In [9]:
import os
from collections import defaultdict
from mp3_sorter_tagger.helpers import check_name_change_safe, split_album_year_name, split_album_name_type

def get_library():
    root = '/mnt/d/Soulseek'
    folders = {}
    for band in os.listdir(root):
        if band == '_VA':
            continue
        folders[band] = {}
        for album in os.listdir(os.path.join(root, band)):
            path = os.path.join(root, band, album)
            if os.path.isdir(path):
                folders[band][album] = os.listdir(path)
    return folders

# folders = get_library()

path = '/home/naztom/tvd/mp3_sorter_tagger/data/'
os.chdir(path)

# to_rename = defaultdict(dict)
# all_good = defaultdict(dict)
# incorrect_title = defaultdict(dict)

# for band in os.listdir():
#     os.chdir(path + band)
#     for folder in os.listdir():
#         year, name = split_album_year_name(folder)
#         name, type, name_original, name_changed = split_album_name_type(name)
#         full = ' - '.join([str(year), name]) if year else name
#         full += f' [{type}]' if type else ''
#         if name_changed and not check_name_change_safe(name, name_original):
#             incorrect_title[band][folder] = full
#         elif folder != full:
#             to_rename[band][folder] = full
#         else:
#             all_good[band][folder] = full

# display(incorrect_title)
# display(to_rename)
# display(all_good)

